In [ ]:
import numpy as np
import tensorflow as tf
import gym
import matplotlib.pyplot as plt
from IPython.display import clear_output

import gym_utils
from gen_raw_data import generate_action

import data_utils
from exp_parameters import ExpParam
from train_vision import create_or_load_vae

In [ ]:
################## SETTINGS #####################
raw_dim = (210, 160, 3)
net_dim = (32*4, 32*3, 3)

x = 0
std = 0

exp_param = ExpParam(
    lat_type="discrete",
    dataset='breakout',
    name_prefix='_BLM'+str(x)+'_STD'+str(std),
    latent=[[32*128, 2]],
    raw_type=tf.uint8,
    raw_dim=raw_dim,
    net_dim=net_dim,  # very close to org aspect ration
    g_std=std,
    learning_rate=0.001,
    rec_loss_multiplier=x,
    max_example=5e6,
    batch_size=256,  # for testing
)

data_set = exp_param.dataset
batch_size = exp_param.batch_size

### DATA
train_iter, test_iter = data_utils.load_data(batch_size, data_set)
ball_col = data_utils.ball_col
ball_loss_multiplier = exp_param.rec_loss_multiplier


In [ ]:
### GENERAL SETUP
try:
    sess.close()
except:
    pass

model_path = 'saved_model/'
model_path += 'breakout_continuous_LAT8_MADE1542627083'

### NETWORK
sess, network, saver = create_or_load_vae(model_path, exp_param=exp_param)
tf.global_variables_initializer().run()


In [ ]:
_, _, data = next(test_iter)
images = data[0]
# masks = data[1] * exp_param.rec_loss_multiplier
masks = data[1]  # TODO: Test that this works! (currently 'exp_param.rec_loss_multiplier' is multiplied twice'

# TODO: Test should use hard sample
[img] = sess.run([network.sum_img], feed_dict={
    network.raw_input: images,
    network.mask_in: masks,
    network.is_training: False
})


In [ ]:
from IPython.display import clear_output

# for i in range(images.shape[0]):
for i in range(5):
    plt.figure(figsize=(8,8))
    plt.imshow(img[i])
    file_name = 'breakout_render_{:05d}.png'.format(i)
    plt.title(file_name)
    plt.savefig('./media/images' + file_name)
    clear_output(wait=True)
    plt.show()
    

In [ ]:
## https://stackoverflow.com/questions/41228209/making-gif-from-images-using-imageio-in-python/45258744
import os
import imageio

png_dir = './media/images'
images = []
for file_name in os.listdir(png_dir):
    if file_name.endswith('.png'):
        file_path = os.path.join(png_dir, file_name)
        images.append(imageio.imread(file_path))
        
imageio.mimsave('./media/gif/movie.gif', images)
